In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
cd /content/drive/MyDrive/Colab Notebooks/DLFinalProject

/content/drive/MyDrive/Colab Notebooks/DLFinalProject


In [14]:
!nvidia-smi

Tue Dec  8 21:37:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    56W / 300W |   3295MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [15]:

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


This is for Installing all requirements needed for our experiments. 

In [16]:
#!pip install -q --force-reinstall transformers
# !pip install -q --force-reinstall git+https://github.com/adapter-hub/adapter-transformers.git
!pip install adapter-transformers
# install datasets
!pip install -q datasets

# Make sure that we have a recent version of pyarrow in the session before we continue - otherwise reboot Colab to activate it
import pyarrow
if int(pyarrow.__version__.split('.')[1]) < 16 and int(pyarrow.__version__.split('.')[0]) == 0:
    import os
    os.kill(os.getpid(), 9)

#!git clone https://github.com/huggingface/transformers
#!python transformers/utils/download_glue_data.py --tasks SST

This code block get all data from allennlp this is a one time activity once data is downloaded same files can be used for each runs 

In [17]:
# !curl --create-dirs -Lo ./data/citation_intent/train.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/citation_intent/train.jsonl
# !curl --create-dirs -Lo ./data/citation_intent/dev.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/citation_intent/dev.jsonl
# !curl --create-dirs -Lo ./data/citation_intent/test.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/citation_intent/test.jsonl

# !curl --create-dirs -Lo ./data/sciie/train.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/sciie/train.jsonl
# !curl --create-dirs -Lo ./data/sciie/dev.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/sciie/dev.jsonl
# !curl --create-dirs -Lo ./data/sciie/test.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/sciie/test.jsonl

In [18]:
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Dict, Optional

from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
from transformers import RobertaTokenizer, RobertaModel
from transformers import AutoTokenizer, EvalPrediction, AutoModelWithHeads, AdapterType
from transformers import (
    Trainer,
    TrainingArguments,
    glue_compute_metrics,
    glue_tasks_num_labels,
    set_seed,
    AdapterType,CONFIG_MAPPING,
    MODEL_WITH_LM_HEAD_MAPPING,
    AdapterArguments,
    AdapterConfig,
    AdapterType,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    HfArgumentParser,
    LineByLineTextDataset,
    PreTrainedTokenizer,
    TextDataset,
    Trainer,
    TrainingArguments, 
    AutoModelForMaskedLM, 
    PfeifferConfig, 
    HoulsbyConfig, 
    InvertibleAdapterConfig
)

from typing import Any, Iterable, List, NewType, Optional, Tuple, Union
import dataclasses
import json
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model_name = "roberta-base"
label_count = {'citation_intent':6, 'sciie':7}
tokenizer = AutoTokenizer.from_pretrained(model_name, max_len=512)

**Task Adapter Config**
This is used for modify task adapter config for our ablation study

In [19]:
@dataclass
class CustomConfig(AdapterConfig):
    """
    The adapter architecture proposed by Pfeiffer et. al., 2020.
    Described in https://arxiv.org/pdf/2005.00247.pdf.
    """

    original_ln_before: bool = True
    original_ln_after: bool = True
    residual_before_ln: bool = True
    adapter_residual_before_ln: bool = False
    ln_before: bool = False
    ln_after: bool = False
    mh_adapter: bool = True
    output_adapter: bool = False
    non_linearity: str = "relu"
    reduction_factor: int = 16
    invertible_adapter: Optional[dict] = InvertibleAdapterConfig(
        block_type="nice", non_linearity="relu", reduction_factor=2
    )

**Trainer Setup**


This block stiches everything and readies a trainer for our execution

1.   Parses file and creates a dataset
2.   Fetches pretrained model
3.   if language adapter to be used then Attaches language Adapter 
4.   Load language adapter from file system which was trained in previous notebook
5.   Attaches a task adapter
6.   Creates a trainer with all datasets and training arguments declared as constants in the code





In [20]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx], device=device) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], device=device)
        return item

    def __len__(self):
        return len(self.labels)

def getDF(filename):
    with open(filename) as f:
        content = f.readlines()
    data = []
    for t in content:
      data.append(json.loads(t))
    df = pd.DataFrame(data)
    return df

def getDataset(dataset='citation_intent'):
    le = LabelEncoder()

    filename = f"data/{dataset}/train.jsonl"
    new_df = getDF(filename)
    new_df['ENCODE_CAT'] = le.fit_transform(new_df['label'])

    filename = f"data/{dataset}/dev.jsonl"
    dev_df = getDF(filename)
    dev_df['ENCODE_CAT'] = le.fit_transform(dev_df['label'])

    filename = f"data/{dataset}/test.jsonl"
    test_df = getDF(filename)
    test_df['ENCODE_CAT'] = le.fit_transform(test_df['label'])
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    train_df =new_df.reset_index(drop=True)
    dev_df =new_df.reset_index(drop=True)
    test_df =test_df.reset_index(drop=True)

    train_encodings = tokenizer(train_df['text'].tolist() , truncation=True, padding=True)
    dev_encodings = tokenizer(dev_df['text'].tolist() , truncation=True, padding=True)
    test_encodings = tokenizer(test_df['text'].tolist(), truncation=True, padding=True)

    train_dataset = CustomDataset(train_encodings, train_df['ENCODE_CAT'].tolist())
    dev_dataset = CustomDataset(dev_encodings, dev_df['ENCODE_CAT'].tolist())
    test_dataset = CustomDataset(test_encodings, test_df['ENCODE_CAT'].tolist())

    return train_dataset, dev_dataset, test_dataset

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

def trainModelwithClassificationHead(model_name, dataset_name="citation_intent", seed=1, adapterName='pfieffer'):

    training_args = TrainingArguments(
    logging_steps=10000, 
    per_device_train_batch_size=4, 
    per_device_eval_batch_size=4, 
    save_steps=20000,
    evaluate_during_training=True,
    output_dir=f"./models/{dataset_name}_{adapterName}",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict=True,
    learning_rate=0.0001,
    num_train_epochs=20,
    seed = seed,
    save_total_limit = 1,
    gradient_accumulation_steps = 1
    )

    train_dataset, dev_dataset, test_dataset = getDataset(dataset=dataset_name)

    if adapterName == 'pfieffer':

        language = 'tapt-pfieffer'
        adapter_dir = '/content/drive/MyDrive/Colab Notebooks/DLFinalProject/TAPT-PF'
        model = AutoModelWithHeads.from_pretrained(model_name)
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model.resize_token_embeddings(len(tokenizer))
        adapter_config = PfeifferConfig()
        model.add_adapter(language, AdapterType.text_lang, config=adapter_config)
        model.from_pretrained(adapter_dir, local_files_only=True)

    elif adapterName == 'houlsby':

        language = 'tapt-houlsby'
        adapter_dir = '/content/drive/MyDrive/Colab Notebooks/DLFinalProject/TAPT_houlsby/TAPT_houlsby'
        model = AutoModelWithHeads.from_pretrained(model_name)
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model.resize_token_embeddings(len(tokenizer))
        adapter_config = HoulsbyConfig(invertible_adapter=InvertibleAdapterConfig(block_type='nice', non_linearity='relu', reduction_factor=2))
        model.add_adapter(language, AdapterType.text_lang, config=adapter_config)
        model.from_pretrained(adapter_dir, local_files_only=True)
    
    elif adapterName == 'no_lang_adapter':

        model = AutoModelWithHeads.from_pretrained(model_name)
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model.resize_token_embeddings(len(tokenizer))


    model.train()

    taskAdapterConfig = CustomConfig()
    model.add_adapter(dataset_name, AdapterType.text_task, config=taskAdapterConfig)
    model.add_classification_head(dataset_name, num_labels=label_count[dataset_name], overwrite_ok=True)
    
    if adapterName != 'no_lang_adapter':
        model.set_active_adapters([language, dataset_name])
    else:
        model.set_active_adapters([dataset_name])
    
    model.train_adapter([dataset_name])
    

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=dev_dataset,
        compute_metrics=compute_metrics,
    )

    print(trainer.args.device)
    trainer.train()
    trainer.evaluate()

    # get test results
    _, _, metrics = trainer.predict(test_dataset)
    with open(f'/content/drive/MyDrive/Colab Notebooks/DLFinalProject/results/results_allTFT2.txt', mode='a') as f:
        f.write(f"Test results: \n {adapterName} \n {dataset_name} \n {metrics} \n\n")
    f.close()

    return metrics['eval_f1']

**Trainer Execution**


This block is our final execution block this will run our code for each defined test test for 5 rounds each and will store the result s into a file with its mean F1 score and standard deviation of our F1 score.

In [ ]:
# results with adapters
import itertools
nRounds = 3
dataset_names = ['citation_intent', 'sciie']
adapter_names = ['pfieffer', 'houlsby', 'no_lang_adapter']
exclude_experiments = [('citation_intent','pfieffer'),('citation_intent','houlsby'),('citation_intent','no_lang_adapter'),('sciie','pfieffer')]
for dn, an in itertools.product(dataset_names, adapter_names):
    if (dn, an) in exclude_experiments:
        continue
    f1 = []
    for i in range(nRounds):
        f1.append(trainModelwithClassificationHead(model_name, 
                                                   dataset_name=dn, 
                                                   seed=i, 
                                                   adapterName=an))
    average = np.mean(f1)
    std = np.std(f1)
    var = np.var(f1)
    with open(f'/content/drive/MyDrive/Colab Notebooks/DLFinalProject/results/results_summaryTFT2.txt', mode='a') as f:
        f.write(f"Test results: \n {an} \n {dn} \n Average Macro F1:{average} \n StdDev:{std} \n Variance:{var}\n\n")
    f.close()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

cuda:0


Step,Training Loss,Validation Loss


In [ ]:
cat results_summaryTFT2.txt